# Gather MITRE ATT&CK detection guidance for techniques
### Input: line-separated text file with list of MITRE ATT&CK technique IDs
### Output: Excel file technique name, number and detection guidance


### Import required libraries. Define filepath.

In [ ]:
from attackcti import attack_client
import json
import numpy
import pandas as pd
import numpy as np
import xlsxwriter

inputFile = '/Users/dan.kaiser/OneDrive - LogRhythm, Inc/Documents/ransomware_module/RyukTechniques' #Enter the full path and file name of the text file containing the MITRE Software items you would like analyzed
outputFile = '/Users/dan.kaiser/OneDrive - LogRhythm, Inc/Documents/ransomware_module/RyukDetectionGuidance.xls' ##Enter the full path and file name of the Excel (xlsx) file that you would like as output of this script

### Load the MITRE ATT&CK Enterprise framework

In [ ]:
lift = attack_client()
%time all_enterprise = lift.get_all_enterprise()

### Read list of techniques from a file

In [ ]:
techniques = []
techniques = [line.rstrip() for line in open(inputFile)]

In [ ]:
for technique in techniques:
    print(technique)

### Generate a list of techniques for each software

In [ ]:
techniqueAnalysis = []
for technique in techniques:
    for t in all_enterprise['techniques']:
        techniqueListItem = {}
        if t['external_references'][0]['external_id'] == technique:
            techniqueListItem['techniqueNumber'] = technique
            techniqueListItem['techniqueName'] = t['name']
            techniqueListItem['DetectionGuidance'] = t['x_mitre_detection']
            techniqueAnalysis.append(techniqueListItem)
print(techniqueAnalysis)

### Load the techique list into a dataframe

In [ ]:
techniques = pd.DataFrame(techniqueAnalysis)

### Confirm the dataframe is populated by reviewing per-tactic counts

In [ ]:
techniques['techniqueNumber'].value_counts()


### Write out the techniques dataframes into an Excel workbook

In [ ]:
with pd.ExcelWriter(outputFile) as writer:
    techniques.to_excel(writer, sheet_name='Techniques')
